## ANOVA

In [ ]:
# 정규성 : 종속 변수는 정규분포를 따른다는 가정
# 등분산성 : 그룹 간의 분산이 동일하다는 가정
# 독립성 : 종속 변수들이 서로 영향을 주고 받으면 안 된다는 가정
# 독립성과 정규성, 등분산성이 충족될 때만 ANOVA 수행
# 따라서 정규성 검정과 등분산성 검정 먼저 진행

#### 독립성 검정

In [ ]:
# 카이제곱 검정은 범주형 데이터 간의 독립성을 평가하는 데 사용
# 군집화된 클러스터를 이용하여 생성한 모델의 독립성 평가에는 Durbin-Watson 통계량이 더 적절

In [ ]:
from statsmodels.stats.stattools import durbin_watson
from statsmodels.graphics.tsaplots import plot_acf

# 모델 적합 후 잔차 계산
# model = 군집화 모델
residuals = model.resid

# Durbin-Watson 통계량 계산
durbin_watson_statistic = durbin_watson(residuals)
print("Durbin-Watson 통계량:", durbin_watson_statistic)

# 잔차의 자기상관 함수(ACF) 그래프
plot_acf(residuals)
plt.show()


In [ ]:
# Durbin-Watson 통계량 확인
# -> Durbin-Watson 통계량이 2에 가까우면 독립성이 있다고 판단, 값이 0 또는 4에 가까우면 자기상관이 존재할 가능성이 높음

# 잔차의 자기상관 함수(ACF) 확인
# ACF를 사용하여 잔차 간의 자기상관을 시각적으로 확인 -> ACF 그래프에서 잔차들이 빠르게 감소하고 0 근처에서 fluctuation이 적다면 독립성이 있다고 판단

#### 정규성 검정

In [ ]:
# 정규성 검정 (Shapiro-Wilk)
# 귀무가설 : 정규분포를 따른다
from scipy.stats import shapiro

# 종속변수의 정규성 검정
shapiro_test = shapiro('feature대입')

# P-value 출력
print("Shapiro-Wilk p-value:", shapiro_test.pvalue)

In [ ]:
# p-value가 유의수준(0.05)보다 작으면 종속 변수가 정규분포를 따르지 않음(정규성 가정 만족x)
# p-value가 작을수록 귀무가설을 기각하고 대립가설을 채택하는 경향
# p-value가 유의수준(0.05)보다 커야 등분산성 가정 만족함

#### 등분산성 검정

In [ ]:
# 각각의 클러스터에 속하는 feature들의 등분산성을 확인하려면
# 각 클러스터에 대해 등분산성 검정을 따로 수행
# -> 해당 클러스터 내에서 feature 분산이 동일한지 여부를 검정할 수 있음

In [ ]:
# 등분산성 검정 - 바틀렛검정
# 귀무가설 : 등분산성의 특징이 있다
from scipy.stats import bartlett

# feature(경제지표)
dependent_variables = df['feature1']

# 클러스터(경기국면)
clusters = df['Cluster'].unique()

# 각 클러스터에 대해 등분산성 검정 수행
for cluster in clusters:
    data_cluster = dependent_variables[df['Cluster'] == cluster]
    bartlett_test = bartlett(data_cluster)
    
    print(f"Cluster {cluster} - Bartlett's p-value:", bartlett_test.pvalue)


In [ ]:
# 등분산성 검정 - 레빈검정
# 귀무가설 : 등분산성의 특징이 있다
from scipy.stats import levene

# feature(경제지표)
dependent_variables = df['feature1']

# 클러스터(경기국면)
clusters = df['Cluster'].unique()

# 각 클러스터에 대해 등분산성 검정 수행
for cluster in clusters:
    data_cluster = dependent_variables[df['Cluster'] == cluster]
    levene_test = levene(data_cluster)
    
    print(f"Cluster {cluster} - Levene's p-value:", levene_test.pvalue)

In [ ]:
# p-value가 유의수준(0.05)보다 작으면 그룹 간의 분산이 동일하지 않음(등분산성 가정 만족 x)
# p-value가 작을수록 귀무가설을 기각하고 대립가설을 채택하는 경향
# p-value가 유의수준(0.05)보다 커야 등분산성 가정 만족함 --> ANOVA수행

#### ANOVA 수행

In [ ]:
# 일원분산분석
from scipy.stats import f_oneway

# 각 클러스터에 해당하는 데이터 가져오기
group_A = df[df['Cluster'] == 'A']['경제지표']
group_B = df[df['Cluster'] == 'B']['경제지표']
group_C = df[df['Cluster'] == 'C']['경제지표']

# ANOVA 수행
f_statistic, p_value = f_oneway(group_A, group_B, group_C)

# 결과 출력
print("F-statistic:", f_statistic)
print("P-value:", p_value)

# # 유의수준 설정
# alpha = 0.05

# # P-value와 유의수준 비교
# if p_value < alpha:
#     print("귀무가설을 기각")
# else:
#     print("귀무가설을 기각하지 못했습니다")


In [ ]:
# ANOVA는 그룹 간의 차이가 통계적으로 유의미한 차이인지 판단
# 분산분석 결과를 통해 각 경제 지표들이 클러스터를 설명하는 데 통계적으로 유의미한지를 나타냄
# p-value가 0.05미만이면 통계적으로 유의